This notebook is result of my reading 
<b> <font color = 'Red'>Hands-on Machine Learning using Scikit-Learn, Tensorflow, Keras by Aurelien Geron</font></b>

I utilized his way of approaching the problem and my way of exploring the options in the Scikit-Learn.

## 1. Objective
Predict each districts median price of the house in California State. Target variable name : `median_house_value`

**Performance Measure**
RMSE : Root Mean Squared Error

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=  pd.read_csv("/kaggle/input/california-housing-prices/housing.csv")
data.head()

In [ ]:
data.describe()

In [ ]:
# Libraries
import plotly.express as px

## 1. Exploratory Data Analysis

In [ ]:
data.info()

<font color = 'Blue'>Null check</font>: Except Total Bedrooms column all the columns doesnt have null

In [ ]:
# Housing Median Age
fig = px.histogram(data, x="housing_median_age", nbins = 30)
fig.show()

<font color = 'Blue'>House Median Age:</font> Almost a good number of houses having the age of above 15 and below 40. Also a good number of houses are having the age of 50+

In [ ]:
# Total Rooms
fig = px.histogram(data, x="total_rooms")
fig.show()

<font color = 'blue'>Total Rooms:</font> Since this data is collected in 1990, the total number of rooms in the district is by average 1800 and max 5000 rooms. very few districts hase more than 10000 rooms.

In [ ]:
# Total Bed Rooms
fig = px.histogram(data, x="total_bedrooms")
fig.show()

<font color = 'Blue'>Total Rooms: </font> Similar to Total number of Rooms, Total number of Bedrooms also less than 1500 for most of the districts. very few districts having more than 3000. It may be developed district.

In [ ]:
# Population
fig = px.histogram(data, x="population")
fig.show()

In [ ]:
# Households
fig = px.histogram(data, x="households")
fig.show()

<font color = 'Blue'>Population and Households</font> Similar to Total Rooms and Total Bedrooms, Population and Households are having the similar range of values.

In [ ]:
# median_income
fig = px.histogram(data, x="median_income")
fig.show()

<font color = 'Blue'>Median Income:</font> Most district people are earning more then 2K and upt0 7K. very few districts has more than 10K income. But also based on the textbook, its capped with some range of values. 

In [ ]:
# median_house_value
fig = px.histogram(data, x="median_house_value")
fig.show()

<font color = 'Blue'>Median House Value:</font> The Median house value is start from 50K to 300K. very few house values are in 500K. It may be near to bay.

In [ ]:
# Ocean Proximity
df = data.groupby('ocean_proximity')['ocean_proximity'].count().reset_index(name = 'count')
fig = px.bar(df, x='ocean_proximity', y='count')
fig.show()

<font color = 'Blue'>Ocean Proximity: </font> High number of houses are in 1 hour distance from Ocean, followed by Inland. these areas might be little cheaper. Near Bay and Ocean is good number of Houses.

In [ ]:
fig = px.scatter(data, x="median_house_value", y="ocean_proximity", color = 'housing_median_age')
fig.show()

In [ ]:
fig = px.scatter(data, x="median_house_value", y="housing_median_age", color = 'ocean_proximity')
fig.show()

<font color='Blue'>Age vs House Value vs Ocean Proximity:</font> 
* Island is having very few houses which is also more than 50+ age and price is also little high.
* Inland house values are very low and its spread in all the ages.
* Near Ocean and 1H Ocean is having spread over all the range either Price or Age.
* Near Bay is costly and its spread over all the age range.

In [ ]:
fig = px.scatter(data, x="longitude", y="latitude", color = 'ocean_proximity')
fig.show()

<font color = 'Blue'>Ocean Proximity Map View:</font>
You can see the Ocean Proximity value in a clear way in the chart.

In [ ]:
fig = px.scatter(data, x="longitude", y="latitude", color = 'median_house_value')
fig.show()

<font color = 'Blue'>House Value:</font>
The price of the houses in the bay area is too high compare to all other places.

## Correlation charts

In [ ]:
df_corr = data.corr()
fig = px.imshow(df_corr)
fig.show()

<font color = 'Blue'>Features Correlation:</font>
Almost most of the features are having negative correlation with Median house value and few features are having positive correlation.

In [ ]:
from pandas.plotting import scatter_matrix
attr = ['median_house_value','median_income','total_rooms','housing_median_age']
scatter_matrix(data[attr], figsize = (15,10))

The previous correlation heatmap and the above scatter chart clearly mentions there is a high correlation between median house value and median income in the district. which means who are all getting good income, they are buying a costly house or because of them the real estate values are going high. Since Real estate is one of the investment option :)

**Data Set Split** 

Here we are spliting the dataset into two parts. 75% would be Train and 25% would be Test dataset. also to get the right combination of data in each features values.

In [ ]:
data["income_cat"] = pd.cut(data["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(data,data['income_cat']):
    strat_train_set = data.loc[train_index]
    strat_test_set = data.loc[test_index]
print(strat_train_set.shape)
print(strat_test_set.shape)
print(data.shape)

## Feature Engineering

It plays a very important role in Machine Learning Model Building. Data is in raw format. even though we had completed the Analysis to understand the data and business we cant say this data is good to feed into a ML Model.

Feature Engineering would convert the raw data into Machine consumable data such as proper matrix after Scaling different range of values into a fixed range of value, filling Null values, Encoding Text values into binary values.

### Creating New Features

<b> How creation of new features would help model?</b>

Model requires a lot of data or features to learn a pattern. 
If current number of features is not enough, we can(if possible) create some more features from existing features. 
also creating new feature would help to increase the accuracy of the model.

In [ ]:
# Creating new features from existing
data['rooms_per_household'] = data['total_rooms'] / data['households']
data['bedrooms_per_room'] = data['total_bedrooms'] / data['total_rooms']
data['population_per_household'] = data['population'] / data['households']

In [ ]:
corr = data.corr()
corr['median_house_value'].sort_values(ascending = False)

* Bedrooms_per_room is having negative correlation
* rooms_per_household is having positive correlation

In [ ]:
fig = px.imshow(corr)
fig.show()

## Data Cleaning

Data with Null values or different ranges or missed values would not acceptable by the model. we need to provide quality data to get a better model with high accuracy. There are multiple options to do the data cleaning. lets explore one by one.

Null Values:
Removing null values from the dataset causes data shortage. so better way is filling the values, if its numerical we can give a try with median or mean value, if its categorical mode value.

Instead of traditional approach like fillna, we are going to try Sklearn's Imputer classes.

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imputer  = SimpleImputer(strategy = 'median')
housing_num = data.drop("ocean_proximity", axis = 1)
median = imputer.fit(housing_num)
X = imputer.transform(housing_num)
housing_tr = pd.DataFrame(X, columns = housing_num.columns, index = housing_num.index)

**Categorical Features**

Generally ML algorithms accepts the numerical values to train or predict. there are multiple ways to convert the categorical values into numerical values. pandas provides pandas.dummies() to convert the categorical values to numerical values by creating new columns and placing yes means 1 and no means 0, male means 1 and female means 0 and so more.

Sklearn provides an easy way to handle this. OnehotEncoder. once its fits with data it remembers the respected values. so it would help to fill the data which is used to predict.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
housing_cat = data[['ocean_proximity']]
cat_encoder = OneHotEncoder()
housing_onehot = cat_encoder.fit_transform(housing_cat)

**Pipelines**

A sequence of actions like imputation, encoding the features, normalizing the values would be arranged and kept it as pipeline before the data feed into the ML algorithms.

Generally this pipelines helps to achieve the data preprocessing activities in a proper flow and it can be useable any time.

In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1) # drop labels for training set
housing_labels = strat_train_set["median_house_value"].copy()

In [ ]:
features = strat_train_set.columns.to_list()

In [ ]:
# Custom Transformers to create some extra features.

from sklearn.base import BaseEstimator, TransformerMixin

# column index
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

housing_num = housing.drop("ocean_proximity", axis=1)
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('attribs_adder', CombinedAttributesAdder()),
        ('std_scaler', StandardScaler()),
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

Using the Pipeline, the abouve code, Imput the missing values especially in number of bedrooms, create 3 features(bedrooms_per_room, rooms_per_household, population_per_household) then normalized the values in different ranges.

In [ ]:
# add the categorical columns with transformer
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        ("cat", OneHotEncoder(), cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing)

## Model Training

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)

print("Predictions:", lin_reg.predict(some_data_prepared))

**Validate Model Performance**

To know the performance of the trained model, how good its generalize the new dataset or unknown dataset, there are two methods Root Mean Squred Error and Mean Absolute Error.

In this problem we are using Root Mean Squared Error.

To learn about RMSE: https://www.statisticshowto.com/probability-and-statistics/regression-analysis/rmse-root-mean-square-error/


In [ ]:
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor().fit(housing_prepared, housing_labels)
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
print(tree_rmse)

## Cross Validation
**Scikit learn's K-Fold Cross validation**

Every time this K-fold cross validation would pick a split of sample test data for example 10%, the remaining 90% would be Train data. Data present in the Train data would not be there in Test data. K- number of times it should be folded. 10 means, 10 times with different trained and tested.

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring = 'neg_mean_squared_error', cv = 10)
tree_rmse_scores = np.sqrt(-scores)
print("Scores : ",str(tree_rmse_scores))
print("Mean : ",str(tree_rmse_scores.mean()))
print("Standard Deviation : ",str(tree_rmse_scores.std()))

In [ ]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring = 'neg_mean_squared_error', cv = 10)
lin_rmse_scores = np.sqrt(-lin_scores)
print("Scores : ",str(lin_rmse_scores))
print("Mean : ",str(lin_rmse_scores.mean()))
print("Standard Deviation : ",str(lin_rmse_scores.std()))

Compared the Linear Regression and Decision Tree Models. Linear Regression is performing better as Decision Tree overfits the training data. So will explore Ensemble model- Random Forest.

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor()
forest.fit(housing_prepared, housing_labels)

forest_scores = cross_val_score(forest, housing_prepared, housing_labels, scoring = 'neg_mean_squared_error', cv = 10)
forest_rmse_scores = np.sqrt(-forest_scores)
print("Scores : ",str(forest_rmse_scores))
print("Mean : ",str(forest_rmse_scores.mean()))
print("Standard Deviation : ",str(forest_rmse_scores.std()))

Random Forest model's result is acceptable

## GridSearchCV

This is one of the option to get the optimal values for Hyperparameters to train the RandomForest Model.

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'n_estimators':[10,30, 50, 70, 100],'max_features':[6,8,10,15]},
    {'bootstrap':[False], 'n_estimators':[3,10],'max_features':[2, 3, 4]}]
forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, 
                           param_grid, 
                           cv = 5, 
                           scoring = 'neg_mean_squared_error', 
                           return_train_score = True)
grid_search.fit(housing_prepared, housing_labels)
print(grid_search.best_params_)

In [ ]:
grid_search.best_estimator_

Apply the GridSearch Result best parameters

Just I have used the exact result of grid search

In [ ]:
st_time = time.time()
forest = RandomForestRegressor(max_features = 8)
forest.fit(housing_prepared, housing_labels)

forest_scores = cross_val_score(forest, housing_prepared, housing_labels, scoring = 'neg_mean_squared_error', cv = 10)
forest_rmse_scores = np.sqrt(-forest_scores)
print("Scores : ",str(forest_rmse_scores))
print("Mean : ",str(forest_rmse_scores.mean()))
print("Standard Deviation : ",str(forest_rmse_scores.std()))
end_time = time.time()
print("Total Time : ",str(round(end_time - st_time,2)))

let me change the max features to 10 and see the result

In [ ]:
import time
st_time = time.time()
forest = RandomForestRegressor(max_features = 15)
forest.fit(housing_prepared, housing_labels)

forest_scores = cross_val_score(forest, housing_prepared, housing_labels, scoring = 'neg_mean_squared_error', cv = 10)
forest_rmse_scores = np.sqrt(-forest_scores)
print("Scores : ",str(forest_rmse_scores))
print("Mean : ",str(forest_rmse_scores.mean()))
print("Standard Deviation : ",str(forest_rmse_scores.std()))
end_time = time.time()
print("Total Time : ",str(round(end_time - st_time,2)))

we can see the difference of timing with 8 and 15 features. so we are using the gridsearch option to get the optimal hyperparameters.

If you <font color = 'orange'>like</font> this kernel and want to <font color = 'orange'>fork</font> plz <font color = 'red'>UPVOTE.</font>

If you have suggestions to improve this kernel plz <font color = 'red'>COMMENT.</font>

<font color = 'orange'>************************ Notebook is under construction ************************</font>